In [ ]:
pip install cvzone opencv-python mediapipe

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
INFO: pip is looking at multiple versions of tensorflow to 

In [20]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceMeshModule import FaceMeshDetector
import numpy as np
import math

# Inisialisasi kamera dan detektor
cap = cv2.VideoCapture(0)
hand_detector = HandDetector(detectionCon=0.7, maxHands=2)
face_detector = FaceMeshDetector(maxFaces=1)

# Load gambar monyet
monyet_default = cv2.imread("data/monyet1.jpeg")
monyet_middle = cv2.imread("data/monyet2.jpeg")
monyet_jempol = cv2.imread("data/monyet5.jpeg")
monyet_senyum = cv2.imread("data/monyet6.jpeg")
monyet_shock = cv2.imread("data/monyet7.jpeg")
monyet_tunjuk_mulut = cv2.imread("data/monyet8.jpeg")

def resize_monyet(monyet, h_target):
    h_monyet, w_monyet, _ = monyet.shape
    if h_monyet > h_target:
        scale = h_target / h_monyet
        new_w = int(w_monyet * scale)
        monyet = cv2.resize(monyet, (new_w, h_target))
    return monyet

# Fullscreen window
cv2.namedWindow("Deteksi Gestur + Ekspresi", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Deteksi Gestur + Ekspresi", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    success, img = cap.read()
    if not success:
        break
    img = cv2.flip(img, 1)

    monyet_tampil = monyet_default
    is_shocked = False
    hands_in_chest = False
    middle_fingers_up = 0
    aha_gesture = False
    mouth_center = None

    # --- Deteksi wajah dan mulut ---
    img, faces = face_detector.findFaceMesh(img, draw=False)
    if faces:
        face = faces[0]
        top_lip = face[13]
        bottom_lip = face[14]
        left_lip = face[78]
        right_lip = face[308]
        mouth_center = ((left_lip[0] + right_lip[0]) // 2, (top_lip[1] + bottom_lip[1]) // 2)

        mouth_open = math.hypot(top_lip[0] - bottom_lip[0], top_lip[1] - bottom_lip[1])
        mouth_width = math.hypot(left_lip[0] - right_lip[0], left_lip[1] - right_lip[1])
        ratio_open = mouth_open / mouth_width if mouth_width != 0 else 0

        if ratio_open > 0.35:
            is_shocked = True

    # --- Deteksi tangan ---
    hands, img = hand_detector.findHands(img, flipType=False)

    h, w, _ = img.shape
    chest_y_min, chest_y_max = int(h * 0.5), h
    chest_x_min, chest_x_max = 0, w

    if hands:
        for hand in hands:
            lm = hand["lmList"]
            palm_x, palm_y = lm[0][0], lm[0][1]

            if chest_y_min < palm_y < chest_y_max and chest_x_min < palm_x < chest_x_max:
                hands_in_chest = True

            fingers = hand_detector.fingersUp(hand)

            if fingers == [0, 0, 1, 0, 0]:
                middle_fingers_up += 1

            if fingers == [0, 1, 0, 0, 0] and is_shocked:
                aha_gesture = True

            if len(hands) == 1 and mouth_center:
                index_tip = hand["lmList"][8]
                distance = math.hypot(index_tip[0] - mouth_center[0], index_tip[1] - mouth_center[1])

                if fingers[1] == 1 and distance < 50:
                    monyet_tampil = monyet_tunjuk_mulut
                elif fingers == [1, 0, 0, 0, 0]:
                    monyet_tampil = monyet_jempol

    # --- Ganti tampilan monyet sesuai kondisi ---
    if aha_gesture:
        monyet_tampil = monyet_senyum
    elif middle_fingers_up == 2:
        monyet_tampil = monyet_middle
    elif is_shocked and hands_in_chest:
        monyet_tampil = monyet_shock

    # Gabungkan hasil dan tampilkan
    h_cam, w_cam, _ = img.shape
    monyet_tampil = resize_monyet(monyet_tampil, h_cam)
    combined = np.hstack((img, monyet_tampil))
    cv2.imshow("Deteksi Gestur + Ekspresi", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1761416192.009910   13382 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1761416192.083027   27588 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 580.65.06), renderer: NVIDIA GeForce RTX 3050 6GB Laptop GPU/PCIe/SSE2
I0000 00:00:1761416192.090835   13382 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
W0000 00:00:1761416192.092125   27580 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761416192.106295   27579 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1761416192.159098   27602 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 580.65.06), renderer: NVIDIA GeForce RTX 3050 6GB Laptop GPU/PCIe/SSE2
W0000 00:00:1761416192.161137   27596 inference_feedback_manager.cc:114] Feedback mana